In [13]:
from dashboard.data_load import load_xena_metadata, load_autoimmune_atlas, load_sorf_df_conformed
import pandas as pd

In [14]:
xena_metadata, xena_transcript_ids = load_xena_metadata()
autoimmune_metadata = load_autoimmune_atlas()

In [15]:
sorf_df = load_sorf_df_conformed()

In [29]:
transcripts = [y for x in sorf_df['transcripts_exact'] for y in x]

In [31]:
selected_expression_tcga = pd.read_parquet('../../cache/xena_app.parq', columns=xena_transcript_ids)
#selected_expression_tcga_ave = selected_expression_tcga.groupby(xena_metadata['dashboard_group']).mean()

In [32]:
selected_expression_tcga

,ENST00000472178,ENST00000392081,ENST00000436891,ENST00000609803,ENST00000304465,ENST00000550418,ENST00000574730,ENST00000586752,ENST00000409544,ENST00000509444,ENST00000379445,ENST00000338458,ENST00000551631,ENST00000449469,ENST00000453722,ENST00000567449,ENST00000367449,ENST00000525546,ENST00000595125,ENST00000596923,ENST00000260803,ENST00000564644,ENST00000512897,ENST00000440410,ENST00000471696,ENST00000503754,ENST00000368892,ENST00000588256,ENST00000496764,ENST00000602719,ENST00000296530,ENST00000563671,ENST00000553772,ENST00000613603,ENST00000627814,ENST00000534155,ENST00000579889,ENST00000555348,ENST00000493381,ENST00000621907,ENST00000539990,ENST00000303391,ENST00000510965,ENST00000354473,ENST00000483836,ENST00000520584,ENST00000461610,ENST00000295566,ENST00000452229,ENST00000429175,...,ENST00000392872,ENST00000374870,ENST00000561815,ENST00000464726,ENST00000554272,ENST00000274376,ENST00000587298,ENST00000258062,ENST00000318225,ENST00000607957,ENST00000366623,ENST00000507708,ENST00000296088,ENST00000518377,ENST00000479107,ENST00000333643,ENST00000518174,ENST00000438525,ENST00000529832,ENST00000490957,ENST00000581913,ENST00000438362,ENST00000355007,ENST00000300134,ENST00000420626,ENST00000338625,ENST00000307275,ENST00000263228,ENST00000293604,ENST00000561208,ENST00000396930,ENST00000369752,ENST00000525005,ENST00000397800,ENST00000575626,ENST00000614715,ENST00000538646,ENST00000381323,ENST00000244321,ENST00000528337,ENST00000524717,ENST00000440467,ENST00000478422,ENST00000382422,ENST00000391419,ENST00000589275,ENST00000510409,ENST00000455566,ENST00000448942,ENST00000613217
GTEX-S4Q7-0003-SM-3NM8M,0.001000,21.791597,0.001,4.390849,0.181005,0.001000,0.001000,4.540968,0.751008,0.001000,0.021000,0.001000,1.100989,2.050961,0.371002,0.220997,3.591036,0.160996,0.921017,0.001,8.621268,0.001000,0.001000,23.250664,26.541905,12.820672,21.181018,0.090999,0.001000,0.001000,0.001000,3.401097,0.210998,0.001000,0.141003,0.001,0.001000,0.001000,0.201005,0.001000,0.001,4.180861,0.350990,1.401042,0.001000,0.340990,0.610981,0.001000,0.061002,12.960954,...,17.421537,0.050999,0.521016,0.001000,6.670986,5.731052,0.460988,5.301130,0.001000,2.281053,0.071002,0.001000,3.900881,0.001000,14.041255,0.650987,28.980287,0.001000,0.521016,0.660990,0.441015,0.001000,0.691020,82.699116,0.451000,3.210950,0.561011,61.019565,0.801014,0.021000,3.060994,47.330557,0.171004,0.001000,0.001000,0.001000,0.001,0.240999,0.381010,1.131001,7.840795,0.850982,0.511002,6.781009,0.110998,0.511002,13.791429,1.041022,2.420995,0.001000
TCGA-19-1787-01,0.001000,75.061437,0.001,1.380987,0.850982,0.001000,0.671007,3.561043,0.001000,0.001000,0.001000,15.051131,2.270956,0.441015,2.910970,0.001000,0.571015,0.171004,2.330921,0.001,11.881129,0.090999,0.271007,44.560700,28.920087,0.001000,32.901923,0.001000,1.161026,0.160996,8.321248,0.580996,0.001000,0.150998,0.090999,0.001,0.001000,0.001000,0.001000,0.001000,0.011,5.731052,0.451000,2.710953,0.001000,0.841013,0.001000,2.471009,0.001000,5.931101,...,8.380870,3.991138,0.001000,0.001000,0.001000,0.751008,0.110998,4.221044,0.071002,4.741028,0.001000,0.001000,4.310933,0.531006,1.441030,13.221372,39.830453,0.001000,0.550991,0.280992,0.371002,0.001000,0.001000,29.762009,0.001000,4.030894,0.410997,31.472066,0.181005,0.350990,0.770999,3.361022,1.200970,0.001000,0.110998,0.001000,0.001,16.260503,0.141003,1.950981,9.760804,0.001000,0.921017,4.680946,0.001000,0.100998,7.420930,1.621018,0.090999,0.001000
TCGA-S9-A7J2-01,0.001000,102.643610,0.001,7.330949,2.201062,0.150998,0.001000,7.621104,0.071002,0.001000,0.021000,6.670986,4.121016,0.340990,11.120887,0.381010,1.420994,0.001000,0.171004,0.001,10.891268,0.001000,6.750994,15.161073,45.440286,0.001000,48.201295,0.001000,6.660821,0.001000,10.280938,0.491002,0.001000,0.361007,0.021000,0.001,0.001000,0.001000,0.350990,0.121002,0.001,7.660826,0.171004,22.111117,0.001000,1.980960,0.001000,1.060982,0.001000,5.991013,...,6.430808,1.471003,0.001000,0.001000,0.121002,3.430931,0.001000,2.791035,0.001000,6.450